In [12]:
# Relevant libraries
import pandas as pd
import scipy.optimize as sc
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm


# Load data from CSV file
file2 = pd.read_csv("../../Output/LowPPMMatrix.csv")

file2=file2[file2['Target PPM']!=150]

# Separate the independent and dependent variables
X = file2.drop('Target PPM', axis='columns')
y = file2.loc[:, 'Target PPM']

def funkEQ(X, a,b,c,d,e,f,g,h,i,j,k,l,m):
    R, H, T = X
    with np.errstate(over='ignore'):
        stuff= a ** ( (-1*R**(b) * (H ** c)) * d + e) * f ** (-1 * H * g + h) * i ** (-1 * T * j + k)  + l * (R*T) + m
    return stuff

p0 = [1] * 13

def residual(params, X, y):
    return np.sum((y - funkEQ(X, *params))**2)

# List to store RMSE values
rmse_list = []

# Loop 100 times
for iteration in tqdm(range(1000)):
    # Split the data into training and testing sets with a fixed random state
    X_train, X_test, y_train, y_test = train_test_split(
        file2[['Resistance', 'RelativeHumidity', 'Temperature']],
        file2['Target PPM']+4.21,
        test_size=0.1,
        random_state=iteration  # The random state is varied with the iteration
    )

    # Perform curve fitting using the funkEQ function and the training data
    popt, pcov = sc.curve_fit(funkEQ, (X_train['Resistance'], X_train['RelativeHumidity'], X_train['Temperature']), y_train, p0, maxfev=1000000)

    # Generate the predictions using the optimized parameters and the test data
    y_pred = funkEQ((X_test['Resistance'], X_test['RelativeHumidity'], X_test['Temperature']), *popt)

    # Calculate the RMSE and store it
    rmse_value = mean_squared_error(y_test + 4.21, y_pred, squared=False)
    rmse_list.append(rmse_value)

# Calculate the average RMSE
average_rmse = np.mean(rmse_list)
print(f'Average RMSE over 1000 iterations: {average_rmse}')


  0%|          | 0/1000 [00:00<?, ?it/s]/Users/benfunk/anaconda3/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/benfunk/anaconda3/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/benfunk/anaconda3/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/benfunk/anaconda3/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/benfunk/anaconda3/lib/python3.10/site

Average RMSE over 1000 iterations: 16.623864802555556


New Funk Equations Data (1000)

a ** ( (-1*R**(b) * (H ** c)) * d + e) * f ** (-1 * H * g + h) * i ** (-1 * T * j + k) * l ** (-1*R*(1/(T*m))*n+o) + p*(R*T) +q
Average RMSE over 100 iterations: 7.007273999664978


a ** ( (-1*R**(b) * (H ** c)) * d + e) * f ** (-1 * H * g + h) * l ** (-1*R*(1/(T*m))*n+o) + p*(R*T) +q
Average RMSE over 1000 iterations: 17.67510700120471

a ** ( (-1*R**(b) * (H ** c)) * d + e) * f ** (-1 * H * g + h) + p*(R*T) +q
Average RMSE over 1000 iterations: 17.675107001818105

a ** ( (-1*R**(b) * (H ** c)) * d + e) * f ** (-1 * H * g + h) * i ** (-1 * T * j + k)  + p*(R*T) +q
Average RMSE over 1000 iterations: 7.880237446949506

a *np.exp( (-1*R**(b) * (H ** c)) * d + e) * f ** (-1 * H * g + h) * i ** (-1 * T * j + k) + l
Average RMSE over 1000 iterations: 17.928474351142512

 Funk Equation Iterations
a*R+b
a*np.exp(-1*R*b+c)+d
a*R**b+c
a*R**b+c*H*(a*R**b+c)+d  (Basically Bastviken)
(a*np.exp(-1*R*b+c)+d)+f*H*(a*np.exp(-1*R*b+c)+d)+g
a*np.exp(-1*R*b+c)+d*np.exp(-1*H*f+g)+h (Funk Equation)
a*np.exp((-1*R*b+c)+(-1*H*d+e))+f 4.2
                                                          W/1000  W/500   UV500   UV1000
a**((-1*R*b+c)+(-1*H*d+e))+f 6.4                          87.79   42.64   62.74   116.35
a**((-1*R*b)+(-1*H*c)+d)+e   6.5                          87.79

a**((((-1*R)/(H**b))*c)+(-1*H*d)+e)+f 7.1                 74.09   35.76   62.37   113.46


a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+f)+g 8.1
a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+(((-1)/(T*f*H**g))*h)+i)+j 8.2
a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+(((-1*T*f)/(H**g))*h)+i)+j 8.3                           5.764

a**((((-1*R)/(H**b))*c)+(-1*H*d)+(-1*T*e)+(((-1*T*f)/(H**g))*h)+i)+j*np.exp(-1*T*k)+l 9.1


0.97**((((-1*R)/(H**(-0.66)))*c)+(-1*H)+(1.21*T)+(((-1.22*T)/(H**0.23))*1.25)+-178.26)+j            5.865


6.33 a ** ( (-1*R**(p) / (H ** b)) * c + d) * e ** (-1 * H * f + g) * h ** (-1 * T * i + j) \
               * k ** (-1*R*(1/(T*l))*m+n) + o

6.18 a ** ( (-1*R**(p) / (H ** b)) * c + d) * e ** (-1 * H * f + g) * h ** (-1 * T * i + j) \
               * k ** (-1*R*(1/(T*l))*m+n) + o*np.log(H) +q

5.75 a ** ( (-1*R**(p) / (H ** b)) * c + d) * e ** (-1 * H * f + g) * h ** (-1 * T * i + j) \
               * k ** (-1*R*(1/(T*l))*m+n) + o*(R*T) +q

